### Import Required Packages

In [ ]:
%load_ext cython

In [ ]:
import os
import sys
import git

import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.3f}'.format)

#### Put the Main Package Library on the PYTHONPATH

In [3]:
git_repo = git.Repo('.', search_parent_directories=True)
git_root = git_repo.git.rev_parse('--show-toplevel')

cython_path = os.path.join(git_root, 'rankfm')
data_path = os.path.join(git_root, "data/examples")

sys.path[0] = git_root
sys.path[1] = cython_path
sys.path[:2]

['/Users/ericlundquist/Repos/rankfm',
 '/Users/ericlundquist/Repos/rankfm/rankfm']

#### Dynamically Re-Load all Package Modules

In [4]:
%load_ext autoreload
%autoreload 2

from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity

#### Re-Compile and Load Cython Functions

In [5]:
!cd $cython_path && cythonize -ia cython_methods.pyx

Compiling /Users/ericlundquist/Repos/rankfm/rankfm/cython_methods.pyx because it changed.
[1/1] Cythonizing /Users/ericlundquist/Repos/rankfm/rankfm/cython_methods.pyx
running build_ext
building 'rankfm.cython_methods' extension
creating /Users/ericlundquist/Repos/rankfm/tmpdoatd_qy/Users
creating /Users/ericlundquist/Repos/rankfm/tmpdoatd_qy/Users/ericlundquist
creating /Users/ericlundquist/Repos/rankfm/tmpdoatd_qy/Users/ericlundquist/Repos
creating /Users/ericlundquist/Repos/rankfm/tmpdoatd_qy/Users/ericlundquist/Repos/rankfm
creating /Users/ericlundquist/Repos/rankfm/tmpdoatd_qy/Users/ericlundquist/Repos/rankfm/rankfm
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/ericlundquist/anaconda3/include -arch x86_64 -I/Users/ericlundquist/anaconda3/include -arch x86_64 -I/Users/ericlundquist/anaconda3/include/python3.7m -c /Users/ericlundquist/Repos/rankfm/rankfm/cython_methods.c -o /Users/ericlundquist/Repos/rankfm/tmpdoa

In [6]:
import cython_methods
dir(cython_methods)

['__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__pyx_unpickle_Enum',
 '__spec__',
 '__test__',
 'assert_finite',
 'fit',
 'np',
 'reg_penalty']

### Load Example Data

#### Load Interactions Data

In [7]:
raw_interactions = pd.read_csv(os.path.join(data_path, 'ML_1M_RATINGS.csv'))[['user_id', 'item_id']]
raw_interactions = raw_interactions.astype(np.int32)
raw_interactions['user_id'] = raw_interactions['user_id'] - 1
raw_interactions['item_id'] = raw_interactions['item_id'] - 1
raw_interactions.head()

,user_id,item_id
0,0,1192
1,0,660
2,0,913
3,0,3407
4,0,2354


In [8]:
np.random.seed(1492)
raw_interactions['random'] = np.random.random(size=len(raw_interactions))
test_pct = 0.25

In [9]:
train_mask = raw_interactions['random'] <  (1 - test_pct)
valid_mask = raw_interactions['random'] >= (1 - test_pct)

interactions_train = raw_interactions[train_mask][['user_id', 'item_id']]
interactions_valid = raw_interactions[valid_mask][['user_id', 'item_id']]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.item_id.unique())
valid_items = np.sort(interactions_valid.item_id.unique())
cold_start_items = set(valid_items) - set(train_items)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

print("train items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("cold-start items: {}".format(cold_start_items))

train shape: (749724, 2)
valid shape: (250485, 2)
train users: 6040
valid users: 6038
cold-start users: set()
train items: 3666
valid items: 3531
cold-start items: {3201, 2562, 643, 3459, 3208, 3721, 138, 395, 650, 1554, 1433, 1914, 1829, 2216, 2217, 686, 1841, 2741, 1850, 2618, 575, 3320, 3322, 3276, 3279, 600, 3289, 3163, 988, 864, 225, 3171, 1385, 877, 3186, 757, 3064, 762, 2555, 126}


### Test Out the Cython Methods

#### Initialize Internal Model Data

In [10]:
model = RankFM(factors=20, loss='warp', max_samples=10, regularization=0.01, sigma=0.1, learning_rate=0.10, learning_schedule='constant')
model._init_all(interactions_train)
model

/Users/ericlundquist/anaconda3/lib/python3.7/site-packages/numba/typed/typeddict.py:34: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  d[key] = value


#### Get Global References to Internal Data for Testing

In [11]:
interactions = np.ascontiguousarray(model.interactions)
sample_weight = np.ascontiguousarray(model.sample_weight)
user_items = model.user_items_py
x_uf = model.x_uf
x_if = model.x_if
w_i = model.w_i
w_if = model.w_if
v_u = model.v_u
v_i = model.v_i
v_uf = model.v_uf
v_if = model.v_if
regularization = 0.01
learning_rate = 0.10
learning_exponent = 0.25
max_samples = 10
epochs = 10

In [12]:
interactions.shape, sample_weight.shape, len(user_items)

((749724, 2), (749724,), 6040)

#### Call the **CYTHON** Fit Function

In [13]:
%%time
cython_methods.fit(
    interactions,
    sample_weight,
    user_items,
    x_uf,
    x_if,
    w_i,
    w_if,
    v_u,
    v_i,
    v_uf,
    v_if,
    regularization,
    learning_rate,
    learning_exponent,
    max_samples,
    epochs
)


training epoch: 0
log likelihood: -425495.375

training epoch: 1
log likelihood: -407225.03125

training epoch: 2
log likelihood: -394094.875

training epoch: 3
log likelihood: -386629.75

training epoch: 4
log likelihood: -382159.6875

training epoch: 5
log likelihood: -378928.25

training epoch: 6
log likelihood: -375838.09375

training epoch: 7
log likelihood: -373967.21875

training epoch: 8
log likelihood: -372646.28125

training epoch: 9
log likelihood: -370733.15625
CPU times: user 10.1 s, sys: 42.8 ms, total: 10.1 s
Wall time: 10.1 s


#### Post-Fit **CYTHON** Diagnostics

In [14]:
u = 0
i = 3177
j = 440

pairwise_utility = 0.0
pairwise_utility += w_i[i] - w_i[j]
pairwise_utility += np.dot(v_i[i] - v_i[j], v_u[u])
pairwise_utility

0.8901866674423218

In [15]:
model.is_fit = True
cy_hit_rate = hit_rate(model, interactions_valid, k=10)
print("Cython Hit Rate: {}".format(cy_hit_rate))

Cython Hit Rate: 0.7878436568400132


#### Call the **NUMBA** Fit Function

In [28]:
%%time
model.fit(interactions_train, epochs=10, verbose=True)

/Users/ericlundquist/Repos/rankfm/rankfm/numba_methods.py:199: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 1d, C))
  d_v_uf = (x_uf[u][p]) * (v_i[i][f] - v_i[j][f] + np.dot(v_if.T[f], x_if[i] - x_if[j]))
/Users/ericlundquist/Repos/rankfm/rankfm/numba_methods.py:207: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 1d, C))
  d_v_if = (x_if[i][q] - x_if[j][q]) * (v_u[u][f] + np.dot(v_uf.T[f], x_uf[u]))



training epoch: 0
log likelihood: -424652.46

training epoch: 1
log likelihood: -405726.04

training epoch: 2
log likelihood: -395820.81

training epoch: 3
log likelihood: -390597.77

training epoch: 4
log likelihood: -386940.95

training epoch: 5
log likelihood: -385182.35

training epoch: 6
log likelihood: -383601.97

training epoch: 7
log likelihood: -382407.5

training epoch: 8
log likelihood: -381946.06

training epoch: 9
log likelihood: -381557.11
CPU times: user 1min 53s, sys: 1.43 s, total: 1min 54s
Wall time: 2min 12s


#### Post-Fit **NUMBA** Diagnostics

In [29]:
nb_hit_rate = hit_rate(model, interactions_valid, k=10)
print("Numba Hit Rate: {}".format(nb_hit_rate))

Numba Hit Rate: 0.7792315336204041


# START CYTHON HELPER FUNCTION TESTING SANDBOX

#### Linear Algebra

In [36]:
%%cython
# cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True

import cython
from scipy.linalg.cython_blas cimport sdot, sgemv
import numpy as np


cdef void add_vv(int n, float *x, float *y, float *out):
    
    cdef int i
    for i in range(n):
        out[i] = x[i] + y[i]
        
        
cdef void sub_vv(int n, float *x, float *y, float *out):
    
    cdef int i
    for i in range(n):
        out[i] = x[i] - y[i]
        
        
cdef float dot_vv(int n, float *x, float *y):
    
    cdef int i
    cdef float res = 0.0
    for i in range(n):
        res += x[i] * y[i]
    return res
    

def process(float[::1] x, float[::1] y):
    """test math functions"""
    
    # declare local variables
    cdef int one = 1, n = x.shape[0]
    cdef float[:] out = np.empty(n, dtype=np.float32)
    cdef float res
    
    # perform a in-place calculation
    sub_vv(n, &x[0], &y[0], &out[0])
    
    # calculate a dot product
    res = dot_vv(n, &x[0], &y[0])
    # res = sdot(&n, &x[0], &one, &y[0], &one)
    
    
    # return results
    return np.asarray(out)
    # return res

In [37]:
x = np.array([1, 2, 3], dtype=np.float32)
y = np.array([3, 6, 9], dtype=np.float32)
print(x)
print(y)

[1. 2. 3.]
[3. 6. 9.]


In [38]:
process(x, y)

array([-2., -4., -6.], dtype=float32)

### Random Number Generation
* rand() gives a random integer `[0, RAND_MAX]`
* RAND_MAX is typically the max 32-bit integer value or `2,147,483,647`

In [41]:
%%cython

cimport cython
from libc.stdlib cimport rand, RAND_MAX
print("RAND_MAX:", RAND_MAX)

@cython.cdivision(True)
cpdef int random_item(int n_items):
    """sample a random item"""
    
    return rand() % n_items


RAND_MAX: 2147483647


In [42]:
# %%time
random_items = pd.Series([random_item(20) for _ in range(int(1e6))])
random_items.value_counts().sort_index()

0     50349
1     49818
2     50137
3     50134
4     50123
5     50057
6     49794
7     49785
8     49837
9     49895
10    49909
11    50204
12    50130
13    49897
14    50217
15    49899
16    50234
17    49784
18    49842
19    49955
dtype: int64

### Try to Get BLAS Wrappers Working

#### Wrap Vector-Vector and Matrix-Vector Dot Product Functions

In [ ]:
%%cython

from scipy.linalg.cython_blas cimport sdot, sgemv

cpdef float dot_vv(float[::1] x, float[::1] y):
    """compute a vector-vector dot product"""
    
    # set the vector length and stride
    cdef int n = x.shape[0], incx = 1, incy = 1
    
    # call the underlying BLAS routine and return the scalar result
    return sdot(&n, &x[0], &incx, &y[0], &incy)


cpdef void dot_mv(float[::1, :] A, float[::1] x, float[::1] y):
    """compute a matrix-vector dot product"""
        
    # set the matrix dimensions
    cdef int m = A.shape[0], n = A.shape[1]
    
    # set the alpha/beta scalars
    cdef float alpha = 1.0, beta = 0.0
    
    # set the x/y stride
    cdef int incx = 1, incy = 1
    
    # call the underlying BLAS routine and store result in [y]
    # NOTE: the matrix [A] must be F-CONTIGUOUS for this to work as written
    # NOTE: it would be much better to figure out how to get this to work C-CONTIGUOUS
    # NOTE: the result is placed into the vector [y] instead of being returned
    sgemv('N', &m, &n, &alpha, &A[0, 0], &m, &x[0], &incx, &beta, &y[0], &incy)




#### Confirm the Accuracy of the Results

##### vector-vector dot product

In [ ]:
x = np.random.randn(50).astype(np.float32)
y = np.random.randn(50).astype(np.float32)

In [ ]:
%%timeit
np.dot(x, y)

In [ ]:
%%timeit
dot_vv(x, y)

##### matrix-vector dot product

In [ ]:
A = np.random.random(10000).reshape((1000, 10)).T.astype(np.float32)
x = np.random.random(1000).astype(np.float32)
y = np.zeros(10, dtype=np.float32)

print(A.shape)
print(x.shape)
print(y.shape)
A.flags

In [ ]:
%%timeit
np.dot(A, x)

In [ ]:
%%timeit
dot_mv(A, x, y)
y

### Test Out User-Item Dictionaries and Search Function

In [42]:
%%cython

cimport cython
from libc.stdlib cimport malloc, free, rand


cdef int lsearch(int item, int *items, int n) nogil:
    """linear search for a given item in a sorted array of items"""
    
    cdef int i
    for i in range(n):
        if item == items[i]:
            return 1
    return 0


cdef int bsearch(int item, int *items, int n) nogil:
    """binary search for a given item in a sorted array of items"""
    
    cdef int lo = 0
    cdef int hi = n - 1
    cdef int md 
    
    while lo <= hi:
        md = int(lo + (hi - lo) / 2)
        if items[md] == item:
            return 1
        elif (items[md] < item):
            lo = md + 1  
        else:
            hi = md - 1
    return 0
        


        

def process(dict p_user_items):
    """create a C array and try to use it in a [nogil] block"""
    
    # declare all C variables
    cdef int i
    cdef int user
    cdef int item
    cdef int random
    cdef int found
    cdef int n_users
    cdef int *n_items
    cdef int **c_user_items
    
    # count the total users and number of items for each user
    n_users = len(p_user_items)
    n_user_items = {user: len(items) for user, items in p_user_items.items()}
    
    # initialize the [n_items] and [c_user_items] C arrays
    n_items = <int*>malloc(n_users * sizeof(int))
    c_user_items = <int**>malloc(n_users * sizeof(int*))
    
    # fill the C arrays
    for user in range(n_users):
        
        # fill the item count array for each user
        n_items[user] = n_user_items[user]
        
        # allocate the dynamic item set array for each user
        c_user_items[user] = <int*>malloc(n_items[user] * sizeof(int))
        
        # fill the item set array for each user
        for item in range(n_items[user]):
            c_user_items[user][item] = p_user_items[user][item]
            
            
    # sample a random negative item and see if it's in that user's items
    for user in range(n_users):
        for i in range(10):
            random = rand() % 10
            found = bsearch(random, c_user_items[user], n_items[user])
            print("user:", user, "item:", random, "found:", found)
        
    # free the dynamic item arrays for each user 
    for user in range(n_users):
        free(c_user_items[user])
        
    # free the item count and top-level user items arrays
    free(n_items)
    free(c_user_items)
        


In [43]:
p_user_items = {
    0: np.sort(np.array([3, 5, 7], dtype=np.int32)),
    1: np.sort(np.array([2, 4, 6, 8, 9], dtype=np.int32)),
    2: np.sort(np.array([4, 5, 6], dtype=np.int32))
}
p_user_items

{0: array([3, 5, 7], dtype=int32),
 1: array([2, 4, 6, 8, 9], dtype=int32),
 2: array([4, 5, 6], dtype=int32)}

In [44]:
process(p_user_items)

user: 0 item: 0 found: 0
user: 0 item: 8 found: 0
user: 0 item: 9 found: 0
user: 0 item: 2 found: 0
user: 0 item: 8 found: 0
user: 0 item: 5 found: 1
user: 0 item: 8 found: 0
user: 0 item: 3 found: 1
user: 0 item: 9 found: 0
user: 0 item: 5 found: 1
user: 1 item: 4 found: 1
user: 1 item: 3 found: 0
user: 1 item: 9 found: 1
user: 1 item: 2 found: 1
user: 1 item: 3 found: 0
user: 1 item: 5 found: 0
user: 1 item: 3 found: 0
user: 1 item: 2 found: 1
user: 1 item: 2 found: 1
user: 1 item: 9 found: 1
user: 2 item: 0 found: 0
user: 2 item: 4 found: 1
user: 2 item: 6 found: 1
user: 2 item: 7 found: 0
user: 2 item: 4 found: 1
user: 2 item: 8 found: 0
user: 2 item: 4 found: 1
user: 2 item: 6 found: 1
user: 2 item: 1 found: 0
user: 2 item: 9 found: 0


# **START EXAMPLE CODE FROM ONLINE RESOURCES**

### Chapter 1 - Cython Essentials

#### Sample Python Program

In [ ]:
def p_fib(n):
    a, b = 0.0, 1.0
    for i in range(n):
        a, b = a + b, a
    return a

#### Equivalent Cython Program

In [ ]:
%%cython --annotate

def c_fib(int n):
    cdef int i
    cdef double a=0.0, b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%%timeit
p_fib(1000)

In [ ]:
%%timeit
c_fib(1000)

### Chapter 9 - Memoryviews

In [ ]:
%%cython

def summer_1(float[:] mv):
    """iterate over a data buffer and compute the sum"""
    
    cdef:
        double d
        ss = 0.0
    for d in mv:
        ss += d
    return ss

#### Faster Version with Direct Access and No Python Overhead
* turn off `boundscheck` and `wraparound`
* use a typed iterator (i) and range(N) for looping
* notice zero python interaction in the function body

In [ ]:
%%cython --annotate

from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
def summer_2(float[:] mv):
    """loop with a typed range and iterator to index directly into the buffer with no Python overhead"""
    
    cdef:
        int i, N
        float ss = 0.0
        
    # easily calculate number of elements
    N = mv.shape[0]
    
    # loop over elements and index directly
    for i in range(N):
        ss += mv[i]
        
    return ss

In [ ]:
to_sum = np.ones(1000000, dtype=np.float32)

In [ ]:
%%timeit
summer_1(to_sum)

In [ ]:
%%timeit
summer_2(to_sum)

In [ ]:
%%cython --annotate

from cython cimport boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
def mv_sum(int[:, ::1] mv):
    """sum the elements of a 2D array"""
    
    cdef int N, M, i, j
    cdef long res = 0
    
    N = mv.shape[0]
    M = mv.shape[1]
    
    for i in range(N):
        for j in range(M):
            res += mv[i,j]
            
    return res

In [ ]:
to_sum = np.ones((100, 100), dtype=np.int32)

In [ ]:
mv_sum(to_sum)

### Bigger Example Using Typed MemoryViews

In [ ]:
%%cython

from array import array
from math import sqrt

from cython cimport cdivision, boundscheck, wraparound
from cpython.array cimport array

@cdivision(True)
cdef inline double A(int i, int j):
    """pulls an element from a predictable infinite matrix"""
    return 1.0 / (((i + j) * (i + j + 1) >> 1) + i + 1)

@boundscheck(False)
@wraparound(False)
cdef void A_times_u(double[::1] u, double[::1] v):
    """matrix-vector dot product - store result in vector [v]"""
    

    cdef int i, j
    cdef u_len = u.shape[0]
    cdef double partial_sum

    for i in range(u_len):
        partial_sum = 0
        for j in range(u_len):
            partial_sum += A(i, j) * u[j]
        v[i] = partial_sum


@boundscheck(False)
@wraparound(False)
cdef void At_times_u(double[::1] u, double[::1] v):
    """matrix-vector dot product on A-transpose"""
    
    cdef:
        int i, j
        u_len = u.shape[0]
        double partial_sum

    for i in range(u_len):
        partial_sum = 0
        for j in range(u_len):
            partial_sum += A(j, i) * u[j]
        v[i] = partial_sum


def B_times_u(u, out, tmp):
    """swapping helper function"""
    
    A_times_u(u, tmp)
    At_times_u(tmp, out)


def spectral_norm(n):
    """compute the final spectral norm"""
    
    u = array("d", [1.0] * n)
    v = array("d", [0.0] * n)
    tmp = array("d", [0.0] * n)

    for _ in range(10):
        B_times_u(u, v, tmp)
        B_times_u(v, u, tmp)

    vBv = vv = 0

    for ue, ve in zip(u, v):
        vBv += ue * ve
        vv  += ve * ve

    return sqrt(vBv / vv)

### Another Example Using Typed MemoryViews

In [ ]:
%%cython --annotate

from cython cimport cdivision, boundscheck, wraparound
import numpy as np
DTYPE = np.intc


cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)


@boundscheck(False)
@wraparound(False)
def compute_seq(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):

    # declare loop ranges as C-int types
    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]
    assert tuple(array_1.shape) == tuple(array_2.shape)

    # create a numpy array to hold results and a memory view to access it
    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    # declare temporary loop variables and loop iterators as C types
    cdef int tmp
    cdef Py_ssize_t x, y

    # fast loops in C filling the values of the result array
    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    # return the result which will be a numpy array
    return result

### Parallel Version
* NOTE: on OSX `gcc` is actually an alias for `clang` which doesn't support OpenMP
* NOTE: you need to set the environment variable $CC to a versioned command for GCC (e.g. gcc-9) to get it to work

In [52]:
os.environ['CC'] = 'gcc-9'

In [53]:
%%cython --compile-args=/openmp --link-args=/openmp --force
cimport cython
cimport openmp

import cython.parallel as cp
from cython.parallel import parallel, prange

import numpy as np
cimport cython

ctypedef fused my_type:
    int
    double
    long long


# We declare our plain c function nogil
cdef my_type clip(my_type a, my_type min_value, my_type max_value) nogil:
    return min(max(a, min_value), max_value)


@cython.boundscheck(False)
@cython.wraparound(False)
def compute_par(my_type[:, ::1] array_1, my_type[:, ::1] array_2, my_type a, my_type b, my_type c):

    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    assert tuple(array_1.shape) == tuple(array_2.shape)

    if my_type is int:
        dtype = np.intc
    elif my_type is double:
        dtype = np.double
    elif my_type is cython.longlong:
        dtype = np.longlong

    result = np.zeros((x_max, y_max), dtype=dtype)
    cdef my_type[:, ::1] result_view = result

    cdef my_type tmp
    cdef Py_ssize_t x, y

    # We use prange here.
    for x in prange(x_max, nogil=True):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

CompileError: command 'gcc-9' failed with exit status 1

In [ ]:
import numpy as np
array_1 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
array_2 = np.random.uniform(0, 1000, size=(3000, 2000)).astype(np.intc)
a = 4
b = 3
c = 9

In [ ]:
%timeit compute_seq(array_1, array_2, a, b, c)

In [ ]:
%timeit compute_par(array_1, array_2, a, b, c)

### Parallel Programming with PRANGE()

In [ ]:
%%cython

# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp

from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

import numpy as np

cdef inline double norm2(double complex z) nogil:
    return z.real * z.real + z.imag * z.imag


cdef int escape(double complex z, double complex c, double z_max, int n_max) nogil:

    cdef:
        int i = 0
        double z_max2 = z_max * z_max

    while norm2(z) < z_max2 and i < n_max:
        z = z * z + c
        i += 1

    return i


@boundscheck(False)
@wraparound(False)
def calc_julia(int resolution, double complex c, double bound=1.5, double z_max=4.0, int n_max=1000):

    cdef:
        double step = 2.0 * bound / resolution
        int i, j
        double complex z
        double real, imag
        int[:, ::1] counts

    counts = np.zeros((resolution+1, resolution+1), dtype=np.int32)

    for i in prange(resolution + 1, nogil=True, schedule='static', chunksize=1):
        real = -bound + i * step
        for j in range(resolution + 1):
            imag = -bound + j * step
            z = real + imag * 1j
            counts[i,j] = escape(z, c, z_max, n_max)

    return np.asarray(counts)

@boundscheck(False)
@wraparound(False)
def julia_fraction(int[:,::1] counts, int maxval=1000):
    cdef:
        int total = 0
        int i, j, N, M
        
    N = counts.shape[0] 
    M = counts.shape[1]

    for i in prange(N, nogil=True):
        for j in range(M):
            if counts[i,j] == maxval:
                total += 1
    return total / float(counts.size)